In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import itertools, time
from numpy.polynomial.chebyshev import chebvander
from numpy.polynomial.laguerre import lagvander
from scipy.stats import norm
from numpy.polynomial.hermite import hermvander
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
sns.set(style="whitegrid", palette="muted", font_scale=1.2)


# N-dimensions

In [11]:
def simulate_gbm_nd(S0, sigma, R, r, T, N_steps, N_paths, seed=None):
    """
    Simulate n-dimensional correlated GBM under the risk-neutral measure
    using antithetic variates.

    Args:
        S0: initial prices, array-like of shape (n,)
        sigma: volatilities, array-like of shape (n,)
        R: correlation matrix, shape (n, n)
        r: risk-free rate
        T: time to maturity
        N_steps: number of time steps
        N_paths: total number of simulation paths (must be even)
        seed: optional random seed

    Returns:
        paths: ndarray of shape (N_paths, N_steps+1, n)
    """
    S0 = np.array(S0, dtype=float)
    sigma = np.array(sigma, dtype=float)
    n = len(S0)

    if seed is not None:
        np.random.seed(seed)
    assert N_paths % 2 == 0, "N_paths must be even for antithetic sampling"

    dt = T / N_steps
    L = np.linalg.cholesky(R)

    paths = np.zeros((N_paths, N_steps+1, n))
    paths[:, 0, :] = S0

    half = N_paths // 2
    drift = (r - 0.5 * sigma**2) * dt

    for t in range(1, N_steps+1):
        Z_small = np.random.normal(size=(half, n))
        Z = np.vstack([Z_small, -Z_small])
        dW = Z @ L.T * np.sqrt(dt)
        diffusion = sigma * dW
        paths[:, t, :] = paths[:, t-1, :] * np.exp(drift + diffusion)

    return paths


def lsm_price_nd(paths, K, r, w, basis_functions, dt, is_call=False):
    """
    Price an American-style option on a weighted sum via the Longstaff - Schwartz method,
    but perform all regressions on the normalized state S/K.
    """
    N_paths, N_steps_plus1, _ = paths.shape
    discount = np.exp(-r * dt)
    w = np.array(w)
    sign = 1 if is_call else -1

    # Value‐matrix: rows=paths, cols=time‐steps
    V = np.zeros((N_paths, N_steps_plus1))
    V[:, -1] = np.maximum(sign * (paths[:, -1, :] @ w - K), 0)

    # Backward induction over all intermediate exercise dates
    for t in range(N_steps_plus1 - 2, 0, -1):
        S_t    = paths[:, t, :]
        payoff = np.maximum(sign * (S_t @ w - K), 0)
        itm    = payoff > 0
        cont   = discount * V[:, t + 1]

        # Non‐ITM simply continuation
        V[~itm, t] = cont[~itm]

        if np.any(itm):
            # 1) extract the raw in‐the‐money sub‐paths
            S_itm = S_t[itm]                     # shape = (N_itm, n)

            # 2) **normalize** by strike
            S_itm_scaled = S_itm / K             # dimensionless input

            # 3) build the regression matrix on S/K
            B = np.vstack([f(S_itm_scaled) for f in basis_functions]).T

            X       = B
            Y       = cont[itm]
            model   = LinearRegression().fit(X, Y)
            cont_fit = model.predict(X)

            # Decide exercise versus continuation
            exercise     = payoff[itm] > cont_fit
            V[itm, t]    = np.where(exercise, payoff[itm], cont[itm])

    # At t=0, compare immediate exercise to continuation
    payoff0 = np.maximum(sign * (paths[:, 0, :] @ w - K), 0)
    cont0   = discount * V[:, 1]
    V0      = np.where(payoff0 > cont0, payoff0, cont0)

    price  = V0.mean()
    stderr = V0.std(ddof=1) / np.sqrt(N_paths)
    return float(price), float(stderr)


def scale_to_unit(x):
    """
    Scale array x to [-1, 1] using its own min and max.
    """
    x = np.asarray(x, dtype=float)
    a, b = x.min(), x.max()
    if b == a:
        return np.zeros_like(x)
    return 2 * (x - a) / (b - a) - 1

def chebyshev_basis_nd(n, degree):
    indices = list(itertools.product(range(degree+1), repeat=n))
    def make_fn(alpha):
        def fn(S):
            feat = None
            for j, power in enumerate(alpha):
                xj = scale_to_unit(S[:, j])
                Tj = chebvander(xj, degree)[:, power]
                feat = Tj if feat is None else feat * Tj
            return feat
        return fn
    return [make_fn(alpha) for alpha in indices]

def laguerre_basis_nd(n, degree):
    indices = list(itertools.product(range(degree+1), repeat=n))
    def make_fn(alpha):
        def fn(S):
            mats = [lagvander(S[:, j], degree)[:, alpha[j]] for j in range(n)]
            feat = mats[0]
            for mat in mats[1:]:
                feat = feat * mat
            return feat
        return fn
    return [make_fn(alpha) for alpha in indices]

def hermite_basis_nd(n, degree):
    indices = list(itertools.product(range(degree+1), repeat=n))
    def make_fn(alpha):
        def fn(S):
            mats = [hermvander(S[:, j], degree)[:, alpha[j]] for j in range(n)]
            feat = mats[0]
            for mat in mats[1:]:
                feat = feat * mat
            return feat
        return fn
    return [make_fn(alpha) for alpha in indices]


We were initially planning on having both sigma at 0.2 and 0.4, but since we saw no point in presenting it too, we chose to simply run with 0.2. This is the reason most of this part of the code is written inside vol_list loops. Furthermore, in order to change the number of assets to 5, simply switch the n to 5. It shall be noted, that we had to ask a friend for computer power(GPU), as our own CPU's did not have sufficient power. Therefore, all the results, will not be shown here, but the code matches. 

In [12]:
if __name__ == "__main__":
    n = 2
    S0 = [40.0]*n
    r = 0.06
    T = 1.0
    N_steps = 50
    N_paths = 100000
    K = 40.0
    w = [1/n]*n
    seed = 2025

    rho = 0.2
    R = np.full((n, n), rho)
    np.fill_diagonal(R, 1.0)

    vol_list = [0.2]
    basis_gens = {
        'Chebyshev': chebyshev_basis_nd,
        'Laguerre':  laguerre_basis_nd,
        'Hermite':   hermite_basis_nd
    }
    degree = 3

    dt = T / N_steps
    for vol in vol_list:
        sigma = [vol]*n
        paths = simulate_gbm_nd(S0, sigma, R, r, T, N_steps, N_paths, seed)

        for name, gen in basis_gens.items():
            basis = gen(n, degree)

            # Start timer
            t0 = time.perf_counter()
            price, stderr = lsm_price_nd(paths, K, r, w, basis, dt=dt, is_call=False)
            # Stop timer
            elapsed = time.perf_counter() - t0

            print(
                f"n={n}, σ={vol:.2f}, Basis={name:9s} : "
                f"Price={price:.4f} ± {stderr:.4f}   "
                f"(computed in {elapsed:.3f} s)"
            )

n=2, σ=0.20, Basis=Chebyshev : Price=1.6609 ± 0.0064   (computed in 3.130 s)
n=2, σ=0.20, Basis=Laguerre  : Price=1.6609 ± 0.0064   (computed in 2.731 s)
n=2, σ=0.20, Basis=Hermite   : Price=1.6609 ± 0.0064   (computed in 4.648 s)


# European Check
The European basket of option should yield a lower price because the time option embedded into the American should be above 0, which pushes the American price above the European. Therefore, if the price is not eqvalent or higher than the European price, we have priced it wrong. This is theory is not gone through in the assignment itself, but it is in place to compute the European check. As this is a closed formula, this could in theory be solved by pen and paper. 

In [3]:
def basket_put_price(S, K, r, sigma, rho, T, weights=None):
    """
    Calculates the price of an European basket-put under the Black-76 scheme

    Parameters:
        S       : Array of stock prices at t0 given as S_i
        K       : Strikeprice, same for all assets
        r       : Risk-free rate (annualized) 
        sigma   : Volatilities σ_i (scalar for uniform or array for individual)
        rho     : correlation matrix ρ (scalar for uniform or array for individual)
        T       : Time to maturity (in years)
        weights : weights for each asset, they are set to equal

    Returns:
        price of a put (float)
    """
    # Konverter inputs til numpy arrays
    S = np.array(S, dtype=float)
    n = len(S)

    # Weights: default to equal and normalize
    if weights is None:
        weights = np.full(n, 1.0 / n)
    else:
        weights = np.array(weights, dtype=float)
        weights = weights / np.sum(weights)

    # Volatiliteter: ensartet eller per aktiv
    if np.isscalar(sigma):
        sigma = np.full(n, sigma)
    else:
        sigma = np.array(sigma, dtype=float)

    # Korrelationsmatrix
    if np.isscalar(rho):
        corr = np.full((n, n), rho)
        np.fill_diagonal(corr, 1.0)
    else:
        corr = np.array(rho, dtype=float)
        if not np.allclose(corr, corr.T):
            raise ValueError("Correlation matrix must be symmetric")

    # Covariance Σ = D · corr · D
    D = np.diag(sigma)
    Sigma = D @ corr @ D

    # Beregn basket-forward og forward-volatilitet
    B0 = np.dot(weights, S)
    FB = B0 * np.exp(r * T)
    sigmaB2 = weights @ Sigma @ weights
    sigmaB = np.sqrt(sigmaB2)

    # Håndter degenererede tilfælde
    if sigmaB <= 0 or T <= 0:
        return max(K - B0, 0) * np.exp(-r * T)

    # Black-76 d1 og d2
    d1 = (np.log(FB / K) + 0.5 * sigmaB2 * T) / (sigmaB * np.sqrt(T))
    d2 = d1 - sigmaB * np.sqrt(T)

    # Price put under Black-76
    put_price = np.exp(-r * T) * (K * norm.cdf(-d2) - FB * norm.cdf(-d1))
    return put_price


if __name__ == "__main__":
    # Eksempelparametre
    K = 40
    r = 0.06
    rho = 0.2
    T = 1.0
    S0 = 40
    sigma_values = [0.2, 0.4]

    for sigma in sigma_values:
        print(f"Results for sigma = {sigma:.2f}:")

        # 2 aktiver
        price_2 = basket_put_price(
            S=[S0, S0],
            K=K,
            r=r,
            sigma=sigma,
            rho=rho,
            T=T
        )
        print(f" European basket-put price (2 assets): {price_2:.4f}")

        # 5 aktiver
        price_5 = basket_put_price(
            S=[S0] * 5,
            K=K,
            r=r,
            sigma=sigma,
            rho=rho,
            T=T
        )
        print(f" European basket-put price (5 assets): {price_5:.4f}\n")


Results for sigma = 0.20:
 European basket-put price (2 assets): 1.4103
 European basket-put price (5 assets): 0.9205

Results for sigma = 0.40:
 European basket-put price (2 assets): 3.7051
 European basket-put price (5 assets): 2.6594



Since the price for all the different basis functions are above the European counterpart, and is not very far from the European price. We will argue, that this price is somewhat fair for the basket

# Convergence Test

We now turn towards convergence test, here we make three difffernt test. Firstly, the $[\texttt{test\_time\_convergence}]$$. This keeps the sample size fixed, but refines the exercise grid $\Delta_t = \frac{T}{N_{\mathrm{steps}}}$ with $N_{\mathrm{steps}} \in [10,25,50,100,200]$ and computes the LSM price at each refinement. We do this as the American option price should converge as the finer grid we use. I.e. we verify that our backward-induction method is consistent and our time‐discretization error vanishes in $N$. 

Secondly, $[\texttt{test\_mc\_convergence}]$. This holds the time grid fixed at $N_{\mathrm{steps}}=50$, but increases the number of simulated paths. Since LSM is a Monte Carlo method, the statistical error should scale like $(N_{\mathrm{paths}}^{-1/2})$. Therefore, the more paths, the more stable our price should become. 

Lastly, we have $[\texttt{test\_basis\_convergence}]$. This keeps both the grid and the sample size consistent, but will vary the polynomial basis degree, $d = 1,\dots,\mathrm{max\_deg}$. Since the regression basis approximates the true continuation-value function, we should see the price stabilize once the basis is sufficiently rich, or might locate eventual overfitting. In order to have the convergence plots for the basis functions, we had to cap it at d=4. 

Can be taken one at a time, or just run the whole script at once. Does take a while.




In [ ]:
basis_gens = {
        'Chebyshev': chebyshev_basis_nd,
        'Laguerre':  laguerre_basis_nd,
        'Hermite':   hermite_basis_nd
    }

In [ ]:
# Parameters
n = 2
S0 = [40.0] * n
sigma = [0.2] * n
rho = 0.2
R = np.full((n, n), rho)
np.fill_diagonal(R, 1.0)
r, T, K = 0.06, 1.0, 40.0
w = [1.0/n] * n
seed = 2025

# Fine‐grid / max‐sample settings
N_steps_max = 200
N_paths_max = 200_000
dt_max = T / N_steps_max

# Simulate once at the finest grid & sample
paths_max = simulate_gbm_nd(S0, sigma, R, r, T, N_steps_max, N_paths_max, seed)

In [ ]:
# Loop over each basis family
for basis_name, gen in basis_gens.items():
    print(f"\n=== Convergence tests using {basis_name} basis ===")

   
    basis3 = gen(n, 3)

    # 1) Time‐step convergence
    time_res = []
    for N_steps in [10, 25, 50, 100, 200]:
        step = N_steps_max // N_steps
        coarse = paths_max[:, ::step, :]
        dt = T / N_steps
        price, _ = lsm_price_nd(coarse, K, r, w, basis3, dt)
        time_res.append((N_steps, dt, price))

    # Plot
    _, dts, prices = zip(*time_res)

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(dts, prices,
            marker='o', linestyle='-',
            linewidth=1.5, markersize=6,
            label='Price')

    # Use a log‐scale on x, but with plain numeric labels
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(ScalarFormatter())
    ax.ticklabel_format(axis='x', style='plain')

    # Grid, labels, title
    ax.grid(True, which='both', linestyle='--', alpha=0.5)
    ax.set_xlabel(r'$\Delta t$ (years)', fontsize=12)
    ax.set_ylabel('American put price', fontsize=12)
    # ax.set_title(f'Time step Convergence ({basis_name})', fontsize=14)
    ax.legend()

    fig.tight_layout()
    plt.show()

    # 2) Monte Carlo convergence
    mc_res = []
    # reuse the same 50-step slicing:
    paths50 = paths_max[:, ::(N_steps_max//50), :]
    dt50 = T / 50
    for N in [10_000, 25_000, 50_000, 100_000, 200_000]:
        subset = paths50[:N]
        price, stderr = lsm_price_nd(subset, K, r, w, basis3, dt50)
        mc_res.append((N, price, stderr))

    # Plot
    Ns, Ps, Es = zip(*mc_res)

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.errorbar(Ns, Ps, yerr=Es,
            fmt='o-', capsize=4,
            linewidth=1.5, markersize=6,
            label='Price ± SE')

    # Log‐scale x with plain numeric labels
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(ScalarFormatter())
    ax.ticklabel_format(axis='x', style='plain')

    # Grid, labels, title
    ax.grid(True, which='both', linestyle='--', alpha=0.5)
    ax.set_xlabel('Number of paths', fontsize=12)
    ax.set_ylabel('American put price', fontsize=12)
    # ax.set_title(f'Monte Carlo Convergence ({basis_name})', fontsize=14)
    ax.legend()

    fig.tight_layout()
    plt.show()
    3) Basis-degree convergence
    basis_res = []
    for degree in [1, 2, 3, 4]:
        basis_d = gen(n, degree)
        price, stderr = lsm_price_nd(paths50, K, r, w, basis_d, dt50)
        basis_res.append((degree, price, stderr))

    # Plot
    Ds, Ps2, Es2 = zip(*basis_res)

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.errorbar(Ds, Ps2, yerr=Es2,
            fmt='o-', capsize=4,
            linewidth=1.5, markersize=6,
            label='Price ± SE')

    ax.grid(True, linestyle='--', alpha=0.5)
    ax.set_xlabel('Polynomial degree', fontsize=12)
    ax.set_ylabel('American put price', fontsize=12)
    # ax.set_title(f'Basis Convergence ({basis_name})', fontsize=14)
    ax.set_xticks(Ds)   # ensure integer tick marks
    ax.legend()

    fig.tight_layout()
    plt.show()



=== Convergence tests using Hermite basis ===


In [ ]:
# print(time_res)
# print(mc_res)
# print(basis_res)

[(10, 0.1, 1.1597454357453172), (25, 0.04, 1.1809563861474892), (50, 0.02, 1.1895097811283462), (100, 0.01, 1.19248222497462), (200, 0.005, 1.1965066774902844)]
[(10000, 1.5153512440403214, 0.0178555105875514), (25000, 1.3263618099064325, 0.010224532006040512), (50000, 1.2527700632804912, 0.0067963973807822955), (100000, 1.214195241219224, 0.004679745813444817), (200000, 1.1895097811283462, 0.0032655591148353044)]
[(1, 1.1456316896674226, 0.0033320906034070103), (2, 1.1699743107141258, 0.0032604371941081464), (3, 1.1895097811283462, 0.0032655591148353044), (4, 1.2334669576193726, 0.003360618795882683)]


# Neural Network

In order to tackle some of our efficiency problems with the rising number of approximations, we will try to incorporate a Neural Network, to see if this will better price the options. Both more accurately and faster. 

We run a setup equivalent to the extended LSM model, but we change the way, we approximate the continuation value. Instead of using regression, we replace it with a network. 

In [ ]:
import numpy as np
from sklearn.neural_network import MLPRegressor

def mlp_lsm(paths_nd, w, r, K, T):
    """
    Price an American‐style basket put via LSM with an n-dimensional MLP,
    and compute its Monte Carlo standard error.

    Parameters
    ----------
    paths_nd : np.ndarray, shape (N_paths, N_steps+1, n_assets)
        Simulated correlated GBM paths.
    w : array-like, shape (n_assets,)
        Basket weights (must sum to 1).
    r : float
        Continuously compounded risk-free rate.
    K : float
        Strike price.
    T : float
        Time to maturity (years).

    Returns
    -------
    price : float
        Estimated option price.
    stderr : float
        Monte Carlo standard error of the estimate.
    """
    N_paths, N_steps1, _ = paths_nd.shape
    N_steps = N_steps1 - 1
    dt = T / N_steps
    df = np.exp(-r * dt)

    # basket price at each time: shape (N_paths, N_steps+1)
    basket = np.tensordot(paths_nd, w, axes=(2, 0))

    # initialize cash flows at maturity (time T)
    cash_flow = np.maximum(K - basket[:, -1], 0)

    # backward induction
    for t in range(N_steps - 1, 0, -1):
        # discount back one step (from t+dt to t)
        cash_flow *= df

        # immediate payoff at t
        immediate = np.maximum(K - basket[:, t], 0)
        itm = immediate > 0

        if np.any(itm):
            # train MLP on in‐the‐money paths
            X = paths_nd[itm, t, :]      # (n_itm, n_assets)
            Y = cash_flow[itm]           # (n_itm,)
            mlp = MLPRegressor(
                hidden_layer_sizes=(40, 40),
                activation='relu',
                solver='adam',
                batch_size=512,
                learning_rate='adaptive',
                learning_rate_init=0.001,
                max_iter=200,
                random_state=42,
            )
            mlp.fit(X, Y)

            # predict continuation for all paths
            cont = mlp.predict(paths_nd[:, t, :])

            # exercise when immediate ≥ continuation
            exercise = itm & (immediate >= cont)
            cash_flow[exercise] = immediate[exercise]

    # final discount from t=dt back to t=0
    discounted = cash_flow * df
    price = np.mean(discounted)
    stderr = np.std(discounted, ddof=1) / np.sqrt(N_paths)
    return price, stderr


In [7]:
if __name__ == "__main__":
    # === Parameters ===
    n       = 5
    S0      = [40.0] * n
    r       = 0.06
    T       = 1.0
    N_steps = 50
    N_paths = 100000
    K       = 40.0
    w       = [1.0 / n] * n
    seed    = 2025
    rho     = 0.2

    # Build correlation matrix
    R = np.full((n, n), rho)
    np.fill_diagonal(R, 1.0)

    # Simulate GBM paths
    paths_nd = simulate_gbm_nd(
        S0=S0,
        sigma=[0.2] * n,
        R=R,
        r=r,
        T=T,
        N_steps=N_steps,
        N_paths=N_paths,
        seed=seed
    )

    # Time the MLP–LSM basket price computation
    t_start = time.perf_counter()
    mlp_price, mlp_error = mlp_lsm(paths_nd, w, r, K, T)
    t_elapsed = time.perf_counter() - t_start

    # Report price and timing
    print(
        f"n={n}, σ=0.20, "
        f"MLP‑LSM (n‑dim) basket price = {mlp_price:.4f} ± {mlp_error:.4f}   "
        f"(computed in {t_elapsed:.3f} s)"
    )

n=5, σ=0.20, MLP‑LSM (n‑dim) basket price = 0.9146 ± 0.0029   (computed in 1457.105 s)
